### Performance remarks

* global vs local scope
* type stability
* memory allocations
* warning premature optimization
* Tuples and Arrays
* type struct fields but not function arguments
* Profile and ProfileView
* Vectorization

In [44]:
n,m = 10000,100
a = rand(n,m)
@time begin
s = 0
for i = 1:n, j=1:m
    global s
    s = s + a[i,j]
end
end
s

  0.481693 seconds (3.04 M allocations: 61.935 MiB, 3.35% gc time)


499868.65660217014

**Always put performance-critical code into functions**

### Type stability

*Use code_warntype to spot type instabilities*

### Cache efficiency

### Memory allocations

### Additional tips

* Use mutating forms of functions
* Use functional form of reductions